In [ ]:
import numpy as np
from scipy import signal
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
def mean_magnitude(d):
    d= np.sqrt(np.sum(d*d, axis=1))
    return np.mean(d)

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def zero_cross(d):
    x = (d[:-1]*d[1:]<0)
    zcr = np.sum(x)
    x = np.argwhere(x==True)
    #print(x.reshape((1,-1)))
    std = np.std(x[1:]-x[:-1])    
    return zcr, std    

In [ ]:
def get_features_one_window(d):
    t, accel, gyro, grav = d[:, 0], d[:, 1:4], d[:, 4:7], d[:, 7:]
    
    accel_mean = np.mean(accel, axis=0)
    gyro_mean = np.mean(gyro, axis=0)
    gyro_mean_mag = mean_magnitude(gyro)
    accel_dv_mean_mag = mean_magnitude(accel[1:, :]-accel[:-1, :])
    
    accel_cov = np.cov(accel, rowvar=False)
    #accel_cov = np.corrcoef(accel, rowvar=False)
    accel_cov = np.array([accel_cov[0,1], accel_cov[0,2], accel_cov[1,2]])
    
    hamming_window = signal.hamming(len(accel))    
    poly_x = np.polyfit(t-t[0], accel[:, 0], deg=4, w=hamming_window)
    poly_y = np.polyfit(t-t[0], accel[:, 1], deg=4, w=hamming_window)
    poly_z = np.polyfit(t-t[0], accel[:, 2], deg=4, w=hamming_window)
    
    high_x = butter_highpass_filter(accel[:, 0], cutoff = 4, fs=16)
    high_y = butter_highpass_filter(accel[:, 1], cutoff = 4, fs=16)
    high_z = butter_highpass_filter(accel[:, 2], cutoff = 4, fs=16)
    
    accel_zcr_x, accel_zcr_std_x = zero_cross(high_x)
    accel_zcr_y, accel_zcr_std_y = zero_cross(high_y)
    accel_zcr_z, accel_zcr_std_z = zero_cross(high_z)
    
    res = np.zeros((1, 32))
    res[0, 0:3] = accel_mean
    res[0, 3:6] = gyro_mean
    res[0, 6] = gyro_mean_mag
    res[0, 7] = accel_dv_mean_mag
    res[0, 8:11] = accel_cov
    res[0, 11:16] = poly_x
    res[0, 16:21] = poly_y
    res[0, 21:26] = poly_z
    res[0, 26] = accel_zcr_x
    res[0, 27] = accel_zcr_y
    res[0, 28] = accel_zcr_z
    res[0, 29] = accel_zcr_std_x
    res[0, 30] = accel_zcr_std_y
    res[0, 31] = accel_zcr_std_z
    
    return res

In [ ]:
def get_features_one_session(data, window_size=5*16, step_size=2):
    count = (len(data)-window_size)//step_size + 1
    res = np.zeros((count, 33))
    
    print("Data sahpe: ", data.shape, "Feature shape: ", res.shape)
    for i in range(count):
        si = i*2        
        d = data[si:si+window_size, :]
        res[i, 0] = d[0, 0]
        res[i, 1:] = get_features_one_window(d)
        if i%10000==0:
            print(i, end=", ")
    
    return res

In [ ]:
def get_features_steven_lab():
    path = 'C:/ASM/DevData/eating_steven/data' if "C:" in os.getcwd() else 'data'    
    feature_path = path + '/features_steven_lab.pkl'
    if os.path.exists(feature_path):
        print("Features file exists. Reading from file...")
        with open(feature_path, 'rb') as file:
            features = pickle.load(file)            
        return features
        
    with open(path+'/data_steven_lab.pkl', 'rb') as file:
        data = pickle.load(file)
    
    features = []    
    for subj in range(len(data)):
        fsubj =[]
        for sess in range(len(data[subj])):
            print("Extracing features... Subj, sess: ", subj, sess)
            d = data[subj][sess]["data_right"]
            f = get_features_one_session(d)
            print("Feature shape: ", f.shape)
            fsubj.append(f)
            
        features.append(fsubj)
        
    with open(feature_path, 'wb') as file:
        pickle.dump(features, file)
        
    return features

In [ ]:
#f=get_features_steven_lab()

In [ ]:
def get_features_steven_free():
    path = 'C:/ASM/DevData/eating_steven/data' if "C:" in os.getcwd() else 'data'    
    feature_path = path + '/features_steven_free.pkl'
    
    if os.path.exists(feature_path):
        print("Features file exists. Reading from file...")
        with open(feature_path, 'rb') as file:
            features = pickle.load(file)            
        return features
        
    with open(path + '/data_steven_free.pkl', 'rb') as file:
        data = pickle.load(file)
    
    features = []    
    for subj in range(len(data)):
        fsubj =[]
        for sess in range(len(data[subj])):
            print("Extracing features... Subj, sess: ", subj, sess)
            d = data[subj][sess]["data_right"]
            f = get_features_one_session(d)
            print("Feature shape: ", f.shape)
            fsubj.append(f)
            
        features.append(fsubj)
        
    with open(feature_path, 'wb') as file:
        pickle.dump(features, file)
        
    return features

In [ ]:
#get_features_steven_free()

In [ ]:
"""
at = data[0][0]["annots"][5, 0]
at = int(at*16)
d = data[0][0]["data_right"]
d = d[at-40:at+40, :]
print(d.shape)
t, accel, gyro, grav = d[:, 0], d[:, 1:4], d[:, 4:7], d[:, 7:]

res = get_features_one_window(d)
print(res.shape)
print(res)
#accel_mean = np.mean(accel, axis=0)
#gyro_mean = np.mean(gyro, axis=0)
#gyro_mean_mag = mean_magnitude(gyro)
#accel_dv_mean_mag = mean_magnitude(accel[1:, :]-accel[:-1, :])
#print(accel_mean)
#print(gyro_mean)
#print(gyro_mean_mag)
#print(accel_dv_mean_mag)

#accel_cov = np.cov(accel, rowvar=False)
#print(accel_cov)
#accel_cov = [accel_cov[0,1], accel_cov[0,2], accel_cov[1,2]]
#print(accel_cov)

#hamming_window = signal.hamming(len(accel))    
#poly_x = np.polyfit(t-t[0], accel[:, 0], deg=4, w=hamming_window)
#poly_y = np.polyfit(t-t[0], accel[:, 1], deg=4, w=hamming_window)
#poly_z = np.polyfit(t-t[0], accel[:, 2], deg=4, w=hamming_window)
#print(poly_x,"\n", poly_y, "\n", poly_z)

#px, py, pz = np.poly1d(poly_x), np.poly1d(poly_y), np.poly1d(poly_z)
#fig,ax=plt.subplots(figsize=(14,8))
#plt.plot(d[:, 0], hamming_window)
#plt.plot(d[:, 0], accel[:, 2])
#plt.plot(d[:, 0], pz(t-t[0]), linestyle=":")
#plt.show()

high_x = butter_highpass_filter(accel[:, 0], cutoff = 4, fs=16)
high_x5 = butter_highpass_filter(accel[:, 0], cutoff = 5, fs=16)
high_x6 = butter_highpass_filter(accel[:, 0], cutoff = 6, fs=16)
high_x3 = butter_highpass_filter(accel[:, 0], cutoff = 3, fs=16)
high_y = butter_highpass_filter(accel[:, 1], cutoff = 3, fs=16)
high_z = butter_highpass_filter(accel[:, 1], cutoff = 3, fs=16)
#accel_zcr, accel_zcr_std = zero_cross(np.concatenate((high_x. high_y, high_z), axis=1))
#accel_zcr, accel_zcr_std = zero_cross(high_y)
#print(accel_zcr)
#print(accel_zcr_std)

fig,ax=plt.subplots(figsize=(14,8))
#plt.plot(d[:, 0], high_x, color='black')
plt.plot((t-t[0])*16, high_y, color='red')
#plt.plot(d[:, 0], high_x5, color='green')
#plt.plot(d[:, 0], high_x6, color='blue')
plt.grid(True)
plt.show()

fig,ax=plt.subplots(figsize=(14,8))
plt.plot(d[:, 0], accel)
plt.show()

fig,ax=plt.subplots(figsize=(14,8))
plt.plot(d[:, 0], gyro)
plt.show()

fig,ax=plt.subplots(figsize=(14,8))
plt.plot(d[:, 0], d[:, 7:])
plt.show()
"""